In [5]:
import wave

# Welcome Audio that has to be played in the starting
with wave.open("../src/callbot/audios/global_ime_3_no.wav", 'rb') as wav_file:
    # Get the number of channels
    channels = wav_file.getnchannels()

    # Get the sample rate
    sample_rate = wav_file.getframerate()

    # Get the sample width
    sample_width = wav_file.getsampwidth()

    # Get the number of frames
    num_frames = wav_file.getnframes()

    # Read the audio data
    audio_data = wav_file.readframes(num_frames)

print(f"Channels: {channels}")
print(f"Sample Rate: {sample_rate} Hz")
print(f"Sample Width: {sample_width} bytes")


Channels: 1
Sample Rate: 24000 Hz
Sample Width: 2 bytes
